In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.9 MB 787.7 kB/s eta 0:00:17
      --------------------------------------- 0.3/12.9 MB 2.0 MB/s eta 0:00:07
     -- ------------------------------------- 0.6/12.9 MB 3.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/12.9 MB 4.5 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/12.9 MB 5.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.9 MB 5.3 MB/s eta 0:00:03
     ------ --------------------------------- 2.2/12.9 MB 5.7 MB/s eta 0:00:02
     ------ --------------------------------- 2.3/12.9 MB 5.5 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.9 MB 5.6 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.9 MB 5.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.9 MB


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/db/e0/52b67d4f00e09e497aec4f71bc44d395605e8ebcea52543242ed34c25ef9/pdfplumber-0.11.7-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     -------------------------------------- 42.8/42.8 kB 345.9 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20250506 from https://files.pythonhosted.org/packages/73/16/7a432c0101fa87457e75cb12c879e1749c5870a786525e2e0f42871d6462/pdfminer_six-20250506-py3-none-any.whl.metadata
  Obtaining dependency information for Pillow>=9.1 from https://files.pythonhosted.org/packages/f1/cc/29c0f5d64ab8eae20f3232da8f8571660aa0ab4b8f1331da5c2f5f9a938e/pillow-11.3.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/25/bd/56d9ec6b9f0fc4e0d95288759f3


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [3]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [14]:
pdf_path = "texts/certificado.pdf"

In [15]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"

In [16]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [17]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [18]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'ilimitada', 'calendario', 'contados', 'fecha', 'atencion', 'comerciante', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'ean', 'n', 'renueve', 'matricula', 'mercantil', 'tardar', 'marzo', 'v', 'i', 'c', 'n', 'n', 's', 'fundamento', 'inscrripciones', 'efectu', 'adas', 'registro', 'mercantil', 'comercio', 'certiefica']


In [20]:
with open("texts/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))